## Instrumental Variables and 2SLS

Today, we are running some code that will help us understand the basics of instrumental variables.
We will analyze the relationship between eating chocolate and happiness. Clearly, we cannot run a simple regression: There may be many omitted variables (e.g., people with lactose intolerance are happier, but they also consume more chocolate) or even reverse causality (e.g., when your GSI is stressed and unhappy, they consume tons of chocolate).

**Thankfully**, we have two potential instrumental variables at hand: 1) We randomly assigned people a voucher that gives them free chocolate, and 2) we know how far they live away from a grocery store.

We will run through the mechanics of the IV estimation. To understand what exactly is going on, we also show you **how the data is generated**, i.e., what the actual **truth** is. This is a trick we can use when we want to check whether a method performs well: We simulate some data, and because we simulated it, we know the truth. Then we can just check whether running a regression with the method we want will give us the correct result.


### Setting up the data

We first load the required packages and set the number of observations (3,000 individuals) and a "seed" - this allows us to use random numbers and get exactly the same numbers every time we run the code.

In [1]:
install.packages("ivreg")
install.packages("huxtable")
install.packages("jtools")

library('ivreg')
library('huxtable')
library('jtools')


set.seed(12345)
n=3000


Installing package into ‘/opt/r’
(as ‘lib’ is unspecified)

Installing package into ‘/opt/r’
(as ‘lib’ is unspecified)

Installing package into ‘/opt/r’
(as ‘lib’ is unspecified)



Next, we generate a data frame and fill it with some observations. The two instruments (voucher and distance) are random variables (one is a "binomial" random variable and will be a dummy, the other a uniform random variable). 

In [2]:
data_iv = data.frame(seq(1, n))
colnames(data_iv)="n"

# The first instrument is a dummy variable: A lottery whether you received a voucher
data_iv$voucher = rbinom(n,1,0.5)

# The second instrument is a continuous variable: The distance to the closest supermarket
data_iv$distance = runif(n,0,1)

Next, we generate some other variables: `unobserved_unhappiness` is how unhappy the respondent was *before* buying any chocolate. We do not observe this and this will generate omitted variable bias (strictly speaking, this is reverse causality).
We also generate a truly random error that is unrelated to anything else in the data.
And we also have data on whether or not a person is lactose intolerant.

In [3]:

# These are some other variables: Being unhappy on a given day, an unobserved error, and a control variable (whether the respondent is lactose intolerant)
data_iv$unobserved_unhappiness = rnorm(n,0,1)
data_iv$yerror = rnorm(n,0,1)
data_iv$lactose_intolerant = rbinom(n,1,0.5)


Finally, we know exactly what determines the consumption of chocolate, and what determines happiness. This is often called the "data-generating process". 


In [4]:

# This is the "data-generating process" for chocolate consumption:
  # - People who got the voucher eat more chocolate
  # People who live further away from supermarket eat less chocolate, people who are lactose intolerant eat less chocolate, people who are currently feeling unhappy eat more chocolate
data_iv$chocolate = 0.8*data_iv$voucher - data_iv$distance - data_iv$lactose_intolerant + data_iv$unobserved_unhappiness 

# This is the DGP for happiness: Eating chocolate makes you happier, being lactose intolerant is also related to happiness, as is being unhappy on a given day, and there is a random error.
data_iv$happiness = data_iv$chocolate + data_iv$lactose_intolerant - data_iv$unobserved_unhappiness + data_iv$yerror


### Questions for you

* Can you see from the DGP: What is the true effect of chocolate on happiness? What would you want to see as regression result?
* Can you guess: If we run the OLS regression (pretending we do not know unobserved unhappiness), if there will be OVB?
* Are distance and voucher valid instruments in this framework (i.e., do they satisfy the relevance, independence, and exclusion restriction)?

### Running OLS

In [5]:

# We immediately see that OLS is biased: unobserved_unhappiness is correlated with chocolate and also with happiness
summary(lm(happiness ~ chocolate + lactose_intolerant, data=data_iv))



Call:
lm(formula = happiness ~ chocolate + lactose_intolerant, data = data_iv)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.0017 -0.7645  0.0156  0.7406  3.8246 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        -0.07876    0.02840  -2.773  0.00558 ** 
chocolate           0.22322    0.01817  12.287  < 2e-16 ***
lactose_intolerant  0.24598    0.04396   5.595  2.4e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.103 on 2997 degrees of freedom
Multiple R-squared:  0.04811,	Adjusted R-squared:  0.04748 
F-statistic: 75.74 on 2 and 2997 DF,  p-value: < 2.2e-16


### IV estimation

We can use the `ivreg` package to use the `voucher` as an instrument for `chocolate` consumption. 

We can also verify that in this simple setup (where the instrument is a dummy variable), we can simply calculate four averages in the data and get **exactly** the same result - so we don't even need to run a regression!

**Cheeky question: Can you come up with at least two reasons why we would still want to run a regression?**

In [6]:

# Runnnig the IV regression
summary(ivreg(happiness ~ chocolate  | voucher , data=data_iv))

# Implementing the Wald estimator
a = mean(data_iv$happiness[data_iv$voucher==1])
  print(a)
b = mean(data_iv$happiness[data_iv$voucher==0])
  print(b)

c = mean(data_iv$chocolate[data_iv$voucher==1])
  print(c)
d = mean(data_iv$chocolate[data_iv$voucher==0])
  print(d)
  
wald_estimator = (a-b)/(c-d)
  print(wald_estimator)
  
  


Call:
ivreg(formula = happiness ~ chocolate | voucher, data = data_iv)

Residuals:
     Min       1Q   Median       3Q      Max 
-5.03580 -1.09931 -0.00718  1.06374  5.32038 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.55613    0.05291   10.51   <2e-16 ***
chocolate    1.09724    0.07583   14.47   <2e-16 ***

Diagnostic tests:
                  df1  df2 statistic p-value    
Weak instruments    1 2998     322.7  <2e-16 ***
Wu-Hausman          1 2997     360.8  <2e-16 ***
Sargan              0   NA        NA      NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.566 on 2998 degrees of freedom
Multiple R-Squared: -0.9222,	Adjusted R-squared: -0.9228 
Wald test: 209.4 on 1 and 2998 DF,  p-value: < 2.2e-16 


[1] 0.3147142
[1] -0.5132467
[1] -0.2200193
[1] -0.9746078
[1] 1.097235


### Two stage least squares

We have seen in class that we can also get the estimate from running two separate regressions and then getting the result as the ration between two OLS coefficients:

In [7]:
# Two-Stage least squares
reduced_form = summary(lm(happiness ~ voucher , data=data_iv))
first_stage  = summary(lm(chocolate ~ voucher , data=data_iv))

tsls = reduced_form$coefficients[2,1] / first_stage$coefficients[2,1]

### Advantages of 2SLS

2SLS gives us several advantages:
* We can use **two instruments at the same time**: distance and voucher. This can help us get more precise estimates because we use more information on what determines chocolate consumption
* We can also **control for additional variables** that are important - such as, in our case, lactose intolerance
* We can directly **test whether instruments are relevant**. This is particularly useful if we have multiple instruments (how would we even do it otherwise?). The way we test this is by looking at the so-called "First stage F-statistic" or here, at the test for "Weak instruments".

In [8]:

summary(a <- ivreg(happiness ~ chocolate  | voucher , data=data_iv))
  # Including Distance as instrument
  summary(b <- ivreg(happiness ~ chocolate  | distance , data=data_iv))
  # Including both instrument
  summary(c <- ivreg(happiness ~ chocolate  | voucher + distance , data=data_iv))
  # Including control
  summary(d <- ivreg(happiness ~ lactose_intolerant + chocolate   | voucher + distance + lactose_intolerant , data=data_iv))

  export_summs(a,b,c,d)


Call:
ivreg(formula = happiness ~ chocolate | voucher, data = data_iv)

Residuals:
     Min       1Q   Median       3Q      Max 
-5.03580 -1.09931 -0.00718  1.06374  5.32038 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.55613    0.05291   10.51   <2e-16 ***
chocolate    1.09724    0.07583   14.47   <2e-16 ***

Diagnostic tests:
                  df1  df2 statistic p-value    
Weak instruments    1 2998     322.7  <2e-16 ***
Wu-Hausman          1 2997     360.8  <2e-16 ***
Sargan              0   NA        NA      NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.566 on 2998 degrees of freedom
Multiple R-Squared: -0.9222,	Adjusted R-squared: -0.9228 
Wald test: 209.4 on 1 and 2998 DF,  p-value: < 2.2e-16 



Call:
ivreg(formula = happiness ~ chocolate | distance, data = data_iv)

Residuals:
      Min        1Q    Median        3Q       Max 
-4.637603 -1.017170 -0.005514  1.014902  4.915180 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.48002    0.05979   8.028 1.41e-15 ***
chocolate    0.96759    0.09118  10.611  < 2e-16 ***

Diagnostic tests:
                  df1  df2 statistic p-value    
Weak instruments    1 2998     185.8  <2e-16 ***
Wu-Hausman          1 2997     143.1  <2e-16 ***
Sargan              0   NA        NA      NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.46 on 2998 degrees of freedom
Multiple R-Squared: -0.6693,	Adjusted R-squared: -0.6698 
Wald test: 112.6 on 1 and 2998 DF,  p-value: < 2.2e-16 



Call:
ivreg(formula = happiness ~ chocolate | voucher + distance, data = data_iv)

Residuals:
       Min         1Q     Median         3Q        Max 
-4.8878490 -1.0666551  0.0000316  1.0406738  5.1698221 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.52785    0.04454   11.85   <2e-16 ***
chocolate    1.04906    0.05921   17.72   <2e-16 ***

Diagnostic tests:
                  df1  df2 statistic p-value    
Weak instruments    2 2997   266.956  <2e-16 ***
Wu-Hausman          1 2997   569.294  <2e-16 ***
Sargan              1   NA     1.191   0.275    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.526 on 2998 degrees of freedom
Multiple R-Squared: -0.8237,	Adjusted R-squared: -0.8243 
Wald test: 313.9 on 1 and 2998 DF,  p-value: < 2.2e-16 



Call:
ivreg(formula = happiness ~ lactose_intolerant + chocolate | 
    voucher + distance + lactose_intolerant, data = data_iv)

Residuals:
     Min       1Q   Median       3Q      Max 
-4.48773 -0.96118 -0.02736  0.98326  4.54457 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        0.004568   0.036602   0.125    0.901    
lactose_intolerant 1.013234   0.072584  13.959   <2e-16 ***
chocolate          1.012618   0.052727  19.205   <2e-16 ***

Diagnostic tests:
                  df1  df2 statistic p-value    
Weak instruments    2 2996   359.423  <2e-16 ***
Wu-Hausman          1 2996   533.505  <2e-16 ***
Sargan              1   NA     1.094   0.296    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.408 on 2997 degrees of freedom
Multiple R-Squared: -0.5516,	Adjusted R-squared: -0.5526 
Wald test: 184.6 on 2 and 2997 DF,  p-value: < 2.2e-16 


Registered S3 methods overwritten by 'broom':
  method            from  
  tidy.glht         jtools
  tidy.summary.glht jtools



,names,Model 1,Model 2,Model 3,Model 4
,<chr>,<chr>,<chr>,<chr>,<chr>
,,Model 1,Model 2,Model 3,Model 4
1,(Intercept),0.556127108534782 ***,0.480022652154977 ***,0.527849971874895 ***,0.00456799144837519
2,,(0.052908777271058),(0.0597934538961056),(0.0445433429512119),(0.0366019688338572)
3,chocolate,1.09723504097503 ***,0.967585383906888 ***,1.04906281499489 ***,1.01261769696456 ***
4,,(0.0758308536003003),(0.0911838574417366),(0.0592114577064265),(0.0527270235605985)
5,lactose_intolerant,,,,1.01323418413147 ***
6,,,,,(0.0725839557382915)
1.1,nobs,3000,3000,3000,3000
2.1,r.squared,-0.922158182071129,-0.669254516745784,-0.823686492940591,-0.551560742341549
